In [ ]:
import os

# Create the main project structure
os.makedirs("src/models", exist_ok=True)
os.makedirs("src/utils", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("results/generated_images", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("data", exist_ok=True)


print("✅ Project directories created successfully!")

✅ Project directories created successfully!


In [ ]:
%%writefile src/config.py
import torch

class Config:
    # --- Data Paths ---
    DATA_ROOT = './data/DIV2K/'
    TRAIN_HR_DIR = DATA_ROOT + 'DIV2K_train_HR/'
    TRAIN_LR_DIR = DATA_ROOT + 'DIV2K_train_LR_bicubic/X4/'

    # --- Model & Training Parameters ---
    SCALE_FACTOR = 4
    IN_CHANNELS = 3
    NUM_RRDB_BLOCKS = 23
    BATCH_SIZE = 32
    NUM_EPOCHS = 100      # You set this to 20
    LEARNING_RATE_G = 1e-4
    LEARNING_RATE_D = 2e-4
    BETA1 = 0.9
    BETA2 = 0.999
    LOG_INTERVAL = 10

    SAVE_INTERVAL = 20 # Save every 20 epochs

    # Add this line back!
    SAVE_EPOCHS = [] # You can leave this empty if you only want to save every SAVE_INTERVAL epochs
                     # Or populate it with specific epochs like [10, 50, 100, 150, 200]
                     # for additional checkpoints.

    # --- Loss Weights ---
    LAMBDA_ADVERSARIAL = 1e-3
    VGG_LAYER_FOR_LOSS = 'features.35'

    # --- Device Configuration ---
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- Paths for Checkpoints, History, and Results ---
    CHECKPOINT_DIR = './checkpoints/'
    HISTORY_FILE = './checkpoints/training_history.npz'
    GENERATED_IMAGES_DIR = './results/generated_images/'
    LOGS_DIR = './results/logs/'

    # --- Paths for Image Generation ---
    GENERATE_LR_DIR = './generate/low_res/'
    GENERATE_HR_DIR = './generate/high_res/'

Writing src/config.py


In [ ]:
%%writefile src/models/generator.py
import torch
import torch.nn as nn

class RRDB(nn.Module):
    def __init__(self, in_channels):
        super(RRDB, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.relu = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.conv3(out)
        return x + out

class Generator(nn.Module):
    def __init__(self, in_channels=3, num_rrdb=23):
        super(Generator, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.rrdb_blocks = nn.Sequential(*[RRDB(64) for _ in range(num_rrdb)])
        self.upsample1 = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.upsample2 = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.final_conv = nn.Conv2d(64, in_channels, kernel_size=3, padding=1)

    def forward(self, x):
        initial_feature = self.relu(self.initial_conv(x))
        out = self.rrdb_blocks(initial_feature)
        out = initial_feature + out
        out = self.upsample1(out)
        out = self.upsample2(out)
        out = self.final_conv(out)
        return out

Writing src/models/generator.py


In [ ]:
%%writefile src/models/discriminator.py
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat, kernel_size=4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(in_channels, 64, normalize=False),
            *block(64, 128),
            *block(128, 256),
            *block(256, 512),
            nn.Conv2d(512, 1, kernel_size=3, stride=1, padding=1)
        )

    def forward(self, img):
        return self.model(img)

Writing src/models/discriminator.py


In [ ]:
%%writefile src/utils/data_loader.py
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class SRDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, transform=None):
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.transform = transform
        self.image_pairs = []

        lr_filenames = sorted([f for f in os.listdir(lr_dir) if f.lower().endswith((".png", ".jpg", ".jpeg"))])

        for lr_fn in lr_filenames:
            hr_fn = lr_fn.replace('x4', '')
            hr_path = os.path.join(self.hr_dir, hr_fn)
            if os.path.exists(hr_path):
                self.image_pairs.append((lr_fn, hr_fn))

        if not self.image_pairs:
            raise FileNotFoundError(f"No matching image pairs found between {lr_dir} and {hr_dir}")

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        lr_fn, hr_fn = self.image_pairs[idx]
        lr_path = os.path.join(self.lr_dir, lr_fn)
        hr_path = os.path.join(self.hr_dir, hr_fn)
        lr_image = Image.open(lr_path).convert("RGB")
        hr_image = Image.open(hr_path).convert("RGB")
        if self.transform:
            lr_image = self.transform['lr'](lr_image)
            hr_image = self.transform['hr'](hr_image)
        return lr_image, hr_image

def get_dataloader(lr_dir, hr_dir, batch_size, shuffle, scale_factor=4, num_workers=2, pin_memory=True):
    HR_IMAGE_SIZE = 256
    LR_IMAGE_SIZE = HR_IMAGE_SIZE // scale_factor
    hr_transform = transforms.Compose([
        transforms.Resize((HR_IMAGE_SIZE, HR_IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    lr_transform = transforms.Compose([
        transforms.Resize((LR_IMAGE_SIZE, LR_IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    transform_dict = {'hr': hr_transform, 'lr': lr_transform}
    dataset = SRDataset(lr_dir=lr_dir, hr_dir=hr_dir, transform=transform_dict)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)

Writing src/utils/data_loader.py


In [ ]:
%%writefile src/utils/losses.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class ContentLoss(nn.Module):
    def __init__(self):
        super(ContentLoss, self).__init__()
    def forward(self, sr, hr):
        return F.mse_loss(sr, hr)

class PerceptualLoss(nn.Module):
    def __init__(self, vgg_layer='features.35', device='cpu'):
        super(PerceptualLoss, self).__init__()
        vgg19 = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features.to(device).eval()
        for param in vgg19.parameters():
            param.requires_grad = False
        self.vgg_features = nn.Sequential(*list(vgg19.children())[:int(vgg_layer.split('.')[-1]) + 1])
    def forward(self, sr, hr):
        sr_features = self.vgg_features(sr)
        hr_features = self.vgg_features(hr).detach()
        return F.mse_loss(sr_features, hr_features)

class DiscriminatorLoss(nn.Module):
    def __init__(self):
        super(DiscriminatorLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()
    def forward(self, real_output, fake_output):
        real_loss = self.bce_loss(real_output, torch.ones_like(real_output))
        fake_loss = self.bce_loss(fake_output, torch.zeros_like(fake_output))
        return real_loss + fake_loss

class GeneratorAdversarialLoss(nn.Module):
    def __init__(self):
        super(GeneratorAdversarialLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()
    def forward(self, fake_output):
        return self.bce_loss(fake_output, torch.ones_like(fake_output))

Writing src/utils/losses.py


In [ ]:
%%writefile src/utils/metrics.py
import torch
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr_metric
from skimage.metrics import structural_similarity as ssim_metric

def calculate_psnr(img1, img2, data_range=1.0):
    if isinstance(img1, torch.Tensor): img1 = img1.detach().cpu().numpy()
    if isinstance(img2, torch.Tensor): img2 = img2.detach().cpu().numpy()
    if img1.ndim == 4:
        psnrs = [psnr_metric(img1[b].transpose(1, 2, 0), img2[b].transpose(1, 2, 0), data_range=data_range) for b in range(img1.shape[0])]
        return np.mean(psnrs)
    return psnr_metric(img1.transpose(1, 2, 0), img2.transpose(1, 2, 0), data_range=data_range)

def calculate_ssim(img1, img2, data_range=1.0):
    if isinstance(img1, torch.Tensor): img1 = img1.detach().cpu().numpy()
    if isinstance(img2, torch.Tensor): img2 = img2.detach().cpu().numpy()
    if img1.ndim == 4:
        ssims = [ssim_metric(img1[b].transpose(1, 2, 0), img2[b].transpose(1, 2, 0), data_range=data_range, channel_axis=-1) for b in range(img1.shape[0])]
        return np.mean(ssims)
    return ssim_metric(img1.transpose(1, 2, 0), img2.transpose(1, 2, 0), data_range=data_range, channel_axis=-1)

Writing src/utils/metrics.py


In [ ]:
# Download HR training images
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip -P ./data/

# Download LR bicubic x4 training images
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip -P ./data/

# Unzip files into the data directory
!unzip -q ./data/DIV2K_train_HR.zip -d ./data/DIV2K/
!unzip -q ./data/DIV2K_train_LR_bicubic_X4.zip -d ./data/DIV2K/

# Clean up zip files
!rm ./data/*.zip

print("✅ Training dataset downloaded and prepared.")

--2025-07-24 17:12:49--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip [following]
--2025-07-24 17:12:49--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3530603713 (3.3G) [application/zip]
Saving to: ‘./data/DIV2K_train_HR.zip’

DIV2K_train_HR.zip  100%[===================>]   3.29G  21.6MB/s    in 2m 38s  

2025-07-24 17:15:28 (21.3 MB/s) - ‘./data/DIV2K_train_HR.zip’ saved [3530603713/3530603713]

--2025-07-24 17:15:28--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip
Resolvi

In [ ]:
%%writefile src/train.py
import torch
import torch.optim as optim
import os
import numpy as np
from torchvision.utils import save_image
import torch.nn.functional as F
from src.models.generator import Generator
from src.models.discriminator import Discriminator
from src.utils.data_loader import get_dataloader
from src.utils.losses import ContentLoss, PerceptualLoss, DiscriminatorLoss, GeneratorAdversarialLoss
from src.utils.metrics import calculate_psnr
from src.config import Config

def train_srgan():
    # --- 1. INITIAL SETUP ---
    config = Config()
    device = config.DEVICE
    start_epoch = 0
    best_psnr = -1.0 # Initialize with a low value for PSNR to ensure first model is saved

    os.makedirs(config.CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(config.GENERATED_IMAGES_DIR, exist_ok=True)
    os.makedirs(config.LOGS_DIR, exist_ok=True)

    # --- 2. INITIALIZE MODELS AND OPTIMIZERS ---
    generator = Generator(in_channels=config.IN_CHANNELS, num_rrdb=config.NUM_RRDB_BLOCKS).to(device)
    discriminator = Discriminator(in_channels=config.IN_CHANNELS).to(device)
    optimizer_G = optim.Adam(generator.parameters(), lr=config.LEARNING_RATE_G, betas=(config.BETA1, config.BETA2))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=config.LEARNING_RATE_D, betas=(config.BETA1, config.BETA2))

    g_loss_history, d_loss_history, psnr_history = [], [], []

    # --- 3. RESUME FROM CHECKPOINT LOGIC ---
    # Find the latest checkpoint (more robust than relying solely on history.npz)
    latest_checkpoint_epoch = -1
    latest_checkpoint_path = None
    for f in os.listdir(config.CHECKPOINT_DIR):
        if f.startswith("checkpoint_epoch_") and f.endswith(".pth"):
            try:
                epoch_num = int(f.replace("checkpoint_epoch_", "").replace(".pth", ""))
                if epoch_num > latest_checkpoint_epoch:
                    latest_checkpoint_epoch = epoch_num
                    latest_checkpoint_path = os.path.join(config.CHECKPOINT_DIR, f)
            except ValueError:
                continue

    if latest_checkpoint_path:
        print(f"=> Loading checkpoint from {latest_checkpoint_path}...")
        try:
            checkpoint = torch.load(latest_checkpoint_path, map_location=device)
            generator.load_state_dict(checkpoint['generator_state_dict'])
            discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
            optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
            optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
            start_epoch = checkpoint['epoch'] + 1 # Start from the next epoch
            g_loss_history = list(checkpoint['g_loss_history'])
            d_loss_history = list(checkpoint['d_loss_history'])
            psnr_history = list(checkpoint['psnr_history'])
            best_psnr = checkpoint.get('best_psnr', -1.0) # Retrieve best_psnr if saved
            print(f"✅ Resuming training from epoch {start_epoch}")
        except Exception as e:
            print(f"❌ Error loading checkpoint {latest_checkpoint_path}: {e}")
            print("Starting training from scratch.")
            start_epoch = 0
            g_loss_history, d_loss_history, psnr_history = [], [], []
            best_psnr = -1.0 # Reset best_psnr if starting fresh
    else:
        print("No checkpoint found. Starting training from scratch.")

    # --- 4. LOSS FUNCTIONS AND DATALOADERS ---
    content_criterion = ContentLoss().to(device)
    perceptual_criterion = PerceptualLoss(vgg_layer=config.VGG_LAYER_FOR_LOSS, device=device)
    discriminator_criterion = DiscriminatorLoss()
    generator_adversarial_criterion = GeneratorAdversarialLoss()

    train_dataloader = get_dataloader(
        lr_dir=config.TRAIN_LR_DIR, hr_dir=config.TRAIN_HR_DIR,
        batch_size=config.BATCH_SIZE, shuffle=True,
        scale_factor=config.SCALE_FACTOR, num_workers=2
    )
    # Using a separate dataloader for sample image to avoid issues with batching
    # when you only need one image.
    sample_dataloader = get_dataloader(
        lr_dir=config.TRAIN_LR_DIR, hr_dir=config.TRAIN_HR_DIR,
        batch_size=1, shuffle=True,
        scale_factor=config.SCALE_FACTOR, num_workers=1 # Using 1 worker for single image
    )
    sample_iterator = iter(sample_dataloader)

    print(f"🚀 Starting training from epoch {start_epoch}...")

    # --- 5. MAIN TRAINING LOOP ---
    for epoch in range(start_epoch, config.NUM_EPOCHS):
        epoch_g_loss = 0.0
        epoch_d_loss = 0.0
        generator.train()
        discriminator.train()

        for i, (lr_images, hr_images) in enumerate(train_dataloader):
            lr_images, hr_images = lr_images.to(device), hr_images.to(device)

            # --- Train Discriminator ---
            optimizer_D.zero_grad()
            real_preds = discriminator(hr_images)
            sr_images = generator(lr_images).detach() # Detach to prevent gradients flowing to G
            fake_preds = discriminator(sr_images)
            d_loss = discriminator_criterion(real_preds, fake_preds)
            d_loss.backward()
            optimizer_D.step()

            # --- Train Generator ---
            optimizer_G.zero_grad()
            sr_images_g = generator(lr_images)
            fake_preds_g = discriminator(sr_images_g)
            c_loss = content_criterion(sr_images_g, hr_images)
            p_loss = perceptual_criterion(sr_images_g, hr_images)
            adv_loss = generator_adversarial_criterion(fake_preds_g)
            g_loss = c_loss + p_loss + config.LAMBDA_ADVERSARIAL * adv_loss
            g_loss.backward()
            optimizer_G.step()

            epoch_g_loss += g_loss.item()
            epoch_d_loss += d_loss.item()

            if (i + 1) % config.LOG_INTERVAL == 0:
                print(f"E[{epoch+1}/{config.NUM_EPOCHS}] S[{i+1}/{len(train_dataloader)}] G_Loss:{g_loss.item():.4f} D_Loss:{d_loss.item():.4f}")

        # --- 6. END OF EPOCH: CALCULATE METRICS AND SAVE ---
        avg_g_loss = epoch_g_loss / len(train_dataloader)
        avg_d_loss = epoch_d_loss / len(train_dataloader)
        g_loss_history.append(avg_g_loss)
        d_loss_history.append(avg_d_loss)
        print(f"End of Epoch {epoch+1} | Avg. G_Loss: {avg_g_loss:.4f} | Avg. D_Loss: {avg_d_loss:.4f}")

        generator.eval()
        with torch.no_grad():
            try:
                sample_lr, sample_hr = next(sample_iterator)
            except StopIteration: # Reset iterator if exhausted
                sample_iterator = iter(sample_dataloader)
                sample_lr, sample_hr = next(sample_iterator)

            sample_lr, sample_hr = sample_lr.to(device), sample_hr.to(device)
            sr_output = generator(sample_lr)

            # Denormalize images for PSNR calculation and saving
            sr_unnorm = (sr_output + 1) / 2
            hr_unnorm = (sample_hr + 1) / 2
            current_psnr = calculate_psnr(sr_unnorm, hr_unnorm, data_range=1.0)
            psnr_history.append(current_psnr)
            print(f"PSNR on sample image: {current_psnr:.4f} dB")

            save_image(sr_unnorm.cpu(), os.path.join(config.GENERATED_IMAGES_DIR, f"epoch_{epoch+1}_sample.png"))

        # --- Checkpoint Saving Logic ---
        # 1. Save every `SAVE_INTERVAL` epochs
        # 2. Save at specific `SAVE_EPOCHS` (from config)
        # 3. Always save the "best" model based on PSNR
        # 4. Always save the history.npz file (as it's used by plot_results)

        # Determine if we should save a periodic checkpoint
        should_save_periodic = (epoch + 1) % config.SAVE_INTERVAL == 0

        # Determine if we should save a specific checkpoint from the list
        should_save_specific = (epoch + 1) in config.SAVE_EPOCHS

        # Save a full checkpoint (models, optimizers, history)
        if should_save_periodic or should_save_specific:
            checkpoint_path = os.path.join(config.CHECKPOINT_DIR, f"checkpoint_epoch_{epoch+1}.pth")
            print(f"✅ Saving checkpoint for epoch {epoch+1}...")
            torch.save({
                'epoch': epoch,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'optimizer_G_state_dict': optimizer_G.state_dict(),
                'optimizer_D_state_dict': optimizer_D.state_dict(),
                'g_loss_history': g_loss_history,
                'd_loss_history': d_loss_history,
                'psnr_history': psnr_history,
                'best_psnr': best_psnr, # Save current best_psnr
            }, checkpoint_path)

        # Save the best model based on PSNR (validation PSNR is usually better here)
        if current_psnr > best_psnr:
            best_psnr = current_psnr
            best_model_path = os.path.join(config.CHECKPOINT_DIR, "best_model.pth")
            print(f"🌟 New best model found! Saving to {best_model_path} with PSNR: {best_psnr:.4f}")
            torch.save({
                'epoch': epoch,
                'generator_state_dict': generator.state_dict(),
                'best_psnr': best_psnr,
            }, best_model_path) # Only save generator and best_psnr for simplicity of "best model"

        # Always save the history file to track progress for resuming plot_results.py
        np.savez(config.HISTORY_FILE,
                 epoch=epoch + 1,
                 g_loss=g_loss_history,
                 d_loss=d_loss_history,
                 psnr=psnr_history)

    print("🏁 Finished Training!")

if __name__ == "__main__":
    train_srgan()

Writing src/train.py


In [ ]:
%%writefile src/plot_results.py
import numpy as np
import matplotlib.pyplot as plt
import os
from src.config import Config

def plot_results():
    config = Config()

    if not os.path.exists(config.HISTORY_FILE):
        print(f"❌ History file not found at {config.HISTORY_FILE}")
        return

    print("=> Loading history to plot graphs...")
    history = np.load(config.HISTORY_FILE)

    g_loss_history = history['g_loss']
    d_loss_history = history['d_loss']
    psnr_history = history['psnr']
    epochs_ran = int(history['epoch'])

    if epochs_ran == 0:
        print("No history found. Please train for at least one epoch.")
        return

    # --- Plot 1: Training Losses ---
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs_ran + 1), g_loss_history, label='Generator Loss', color='blue')
    plt.plot(range(1, epochs_ran + 1), d_loss_history, label='Discriminator Loss', color='red')
    plt.title('Training Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Average Loss')
    plt.legend()
    plt.grid(True)

    # --- Plot 2: PSNR ---
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs_ran + 1), psnr_history, label='PSNR on Sample Image', color='green')
    plt.title('PSNR')
    plt.xlabel('Epochs')
    plt.ylabel('PSNR (dB)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plot_path = os.path.join(config.LOGS_DIR, 'performance_graphs.png')
    plt.savefig(plot_path)
    print(f"✅ Graphs saved to {plot_path}")
    plt.show()

if __name__ == '__main__':
    plot_results()

Writing src/plot_results.py


In [ ]:
!python -m src.train

No checkpoint found. Starting training from scratch.
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100% 548M/548M [00:03<00:00, 150MB/s]
🚀 Starting training from epoch 0...
E[1/100] S[10/25] G_Loss:0.5734 D_Loss:0.7654
E[1/100] S[20/25] G_Loss:0.5494 D_Loss:0.4644
End of Epoch 1 | Avg. G_Loss: 0.6042 | Avg. D_Loss: 0.7120
PSNR on sample image: 14.7916 dB
🌟 New best model found! Saving to ./checkpoints/best_model.pth with PSNR: 14.7916
E[2/100] S[10/25] G_Loss:0.5329 D_Loss:0.2031
E[2/100] S[20/25] G_Loss:0.4590 D_Loss:0.1835
End of Epoch 2 | Avg. G_Loss: 0.5124 | Avg. D_Loss: 0.2143
PSNR on sample image: 19.7775 dB
🌟 New best model found! Saving to ./checkpoints/best_model.pth with PSNR: 19.7775
E[3/100] S[10/25] G_Loss:0.4985 D_Loss:0.1114
E[3/100] S[20/25] G_Loss:0.4661 D_Loss:0.0829
End of Epoch 3 | Avg. G_Loss: 0.4874 | Avg. D_Loss: 0.1088
PSNR on sample image: 21.0533 dB
🌟 New best model found! Saving